In [0]:
!pip install pennylane-sf 
import os
os.kill(os.getpid(), 9)

     |████████████████████████████████| 245kB 4.0MB/s 
     |████████████████████████████████| 3.0MB 9.0MB/s 
     |████████████████████████████████| 6.7MB 19.4MB/s 
     |████████████████████████████████| 5.8MB 38.9MB/s 
     |████████████████████████████████| 133kB 52.5MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 112kB 53.0MB/s 
ERROR: kaggle 1.5.6 has requirement urllib3<1.25,>=1.21.1, but you'll have urllib3 1.25.9 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [27]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer,NesterovMomentumOptimizer,AdagradOptimizer,RMSPropOptimizer
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.metrics import explained_variance_score as evs
from sklearn.metrics import mean_absolute_error as mae
dev = qml.device('strawberryfields.fock', wires=1, cutoff_dim=18,hbar=2)
dev.analytic


True

In [0]:
def layer(v):
  qml.Rotation(v[0], wires=0)
  qml.Squeezing(v[1], 0, wires=0)
  qml.Rotation(v[2], wires=0)
  qml.Displacement(v[3], 0, wires=0)
  qml.Kerr(v[4], wires=0)

In [0]:
@qml.qnode(dev)
def quantum_neural_net(var, x=None):
    
    # Encode input x into quantum state
    qml.DisplacedSqueezedState(x[0], 0,x[1],0, wires=0)

    for v in (var):
        layer(v)

    return qml.expval(qml.X(0))

In [0]:
@qml.qnode(dev)
def trace(var, x=None):
    qml.DisplacedSqueezedState(x[0],0,x[1],0,wires=0)
    for v in var:
        layer(v)
    return qml.expval(qml.Identity([0]))

In [0]:
def cost(var, features, labels):

    out = 0

    for i in range(len(features)):
        preds = quantum_neural_net(var,x=features[i])
        out = out + (preds-labels[i])**2
    
    traces = [trace(var, x=x) for x in features]
    traces = (1-abs(np.array(traces))).mean()

    return (out/len(features))+0.01*traces

In [32]:
url = 'https://raw.githubusercontent.com/kareem1925/arabic_numbers/master/k.csv'
data = pd.read_csv(url)

features = data.drop('I', axis=1).values
I = data.I.values
#I_new = MinMaxScaler(feature_range=(-1,1)).fit_transform(I.reshape(-1,1))
I = data.I.values*1000
#print(I.describe())
#print(abs(I).min())
X_train, X_test, y_train, y_test = train_test_split(features,I,
                                                    test_size=0.15,random_state=0)

Volt = X_train[:,0:1]
Inten = X_train[:,1:]

disp_scale = MinMaxScaler(feature_range=(-1.1,1)).fit(Volt)

squeeze_scale = MinMaxScaler(feature_range=(0,0.8)).fit(Inten)

X_train[:,0:1] = disp_scale.transform(X_train[:,0:1])
X_train[:,1:] = squeeze_scale.transform(X_train[:,1:])
X_test[:,0:1] = disp_scale.transform(X_test[:,0:1])
X_test[:,1:] = squeeze_scale.transform(X_test[:,1:])
#print(np.sum(X_train[:,0]== 0.0))
#
X_train[5],y_train[5],abs(I).min()

(array([0.90869565, 0.8       ]), 1.02944, 0.000407583)

In [0]:
def accuracy(labels, predictions):
    score = mse(labels,predictions)
    s = mae(labels,predictions)
    r =r2(labels,predictions)
    return score,s,r

In [0]:
np.random.seed(1220)
var_init = np.random.normal(0,0.001,(8,5))
var = var_init
opt = AdamOptimizer(0.01, beta1=0.9, beta2=0.999)
opt.reset()
import time

In [0]:
from sklearn.utils import shuffle
old_acc = - np.inf
update_var = var
var_init
#cost(var=var,features=X_train[:2],labels=y_train[:2])
opt

def testing(var,features):
    result = []
    for i in range(len(features)):
        preds = quantum_neural_net(var,x=features[i])
        result.append(preds)
    return np.array(result).reshape(-1,1)

def next_batch(X, y, batchSize):
    rem = len(X)//batchSize
    out = len(X)%batchSize
    # loop over our dataset `X` in mini-batches of size `batchSize`
    for id ,i in enumerate(np.arange(0, X.shape[0], batchSize)):
        # yield a tuple of the current batched data and labels
        if id == rem and out is not 0:
            temp = abs(batchSize-out)
            d = X[i:i + batchSize]
            l = y[i:i + batchSize]
            batch = np.random.choice(622,temp,replace=False)
            
            dd,ll = X[batch],y[batch]

            return (np.vstack((d,dd)),np.vstack((l.reshape(-1,1),ll.reshape(-1,1))))
        yield (X[i:i + batchSize], y[i:i + batchSize])

In [36]:
epochLoss = []
lossHistory = []
RR=[]
R = []
lossHistory_v = []
RR_v=[]
R_v = []
epoch_v = []
W = []
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
weight_epoch=[]
train = []
test = []
e = []    
mse_t = []
mse_v = []
tr=[]
for it in range(10700):
    
    now = time.time()
    batch_test = np.random.choice(X_test.shape[0],X_test.shape[0],replace=False)
    X_test1 = X_test[batch_test]
    y_test1 = y_test[batch_test]
  
    batch = np.random.choice(X_train.shape[0],X_train.shape[0],replace=False)
    feats_train_11 =X_train[batch]
    Y_train1 = y_train[batch]
    
 #   feats_train_11,Y_train1 = shuffle(X_train,y_train)
    for id,(batchX, batchY) in enumerate(next_batch(feats_train_11, Y_train1,32)):
        
        var = opt.step(lambda v: cost(v, batchX, batchY), var)
        e.append(var)
        if id == 1:
            break
        
    if it > 100:
      np.save('weights9',np.array(e))    
        


    t=trace(var,x=X_train[5])
    print("Trace: {:0.5f}".format(t))   
    
    predictions = testing(var,X_test1)
    ww = accuracy(y_test1, predictions)
    acc_test = ww[2]
    acc_m = ww[0]
    weight_epoch.append(var)
    np.save('weights_per_epoch',np.array(weight_epoch))

    test.append(acc_test)
    np.save('testing',test)
    mse_v.append(acc_m)
    
    if acc_test > old_acc:
        update_var = var
        old_acc = acc_test
        print("  ")
        print('model improved')
        print("  ")
    
    print("Iter:{:5d} | mse: {:0.18e} | mae: {:0.18e} | R2: {:0.6f}".format(it + 1,ww[0],ww[1],ww[2]),'time:', int((time.time()-now)))

Trace: 0.99985
  
model improved
  
Iter:    1 | mse: 7.913816877857398069e-01 | mae: 7.493045107526957738e-01 | R2: -11.136052 time: 69
Trace: 0.99984
  
model improved
  
Iter:    2 | mse: 2.401651741540786478e-01 | mae: 4.227998323423743376e-01 | R2: -2.682998 time: 69
Trace: 0.99983
  
model improved
  
Iter:    3 | mse: 8.588933247858135389e-02 | mae: 2.098582110524263022e-01 | R2: -0.317136 time: 68
Trace: 0.99982
Iter:    4 | mse: 2.200846086380871536e-01 | mae: 2.902348279083872828e-01 | R2: -2.375057 time: 69
Trace: 0.99981
Iter:    5 | mse: 2.533144806671466798e-01 | mae: 3.130112787862571744e-01 | R2: -2.884646 time: 68
Trace: 0.99980
Iter:    6 | mse: 1.703939728359878947e-01 | mae: 2.654803193827595664e-01 | R2: -1.613038 time: 68
Trace: 0.99979
  
model improved
  
Iter:    7 | mse: 7.717312647756653654e-02 | mae: 2.131289185892355498e-01 | R2: -0.183471 time: 67
Trace: 0.99979
Iter:    8 | mse: 7.789295413100132126e-02 | mae: 2.417357193580499941e-01 | R2: -0.194510 time

KeyboardInterrupt: ignored